In [1]:
import os

In [2]:
# enter your api key here
os.environ['OPENAI_API_KEY']=''

In [3]:
!pip install openai PyPDF2 langchain tiktoken faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00


In [4]:
files=[
    '/content/gdrive/My Drive/AI paper/data/vaccine_information_statement.pdf',
    '/content/gdrive/My Drive/AI paper/data/update-28.pdf',
    '/content/gdrive/My Drive/AI paper/data/standing-orders.pdf'
]

In [5]:
import openai
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub

In [6]:
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

In [7]:
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [8]:
def get_vectorstore(text_chunks):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

In [9]:
def get_conversation_chain(vectorstore):
    llm = ChatOpenAI(temperature=0.8)
    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

In [10]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
raw_text = get_pdf_text(files)
text_chunks = get_text_chunks(raw_text)
vectorstore = get_vectorstore(text_chunks)


In [12]:
chain=get_conversation_chain(vectorstore)

In [13]:
query = "What is the Countermeasures Injury Compensation Program?"
result = chain({"question": query})
result["answer"]

'The Countermeasures Injury Compensation Program (CICP) is a federal program that may help pay for the costs of medical care and other specific expenses for individuals who have been seriously injured by certain medicines or vaccines, including the COVID-19 vaccine. To learn more about this program, you can visit www.hrsa.gov/cicp or call 1-855-266-2427.'

In [15]:
query="Tell me a way to get a vaccination for COVID-19"
result = chain({"question": query})
result["answer"]

'To obtain a COVID-19 vaccination, you can take the following steps:\n\n1. Ask your vaccination provider for more information.\n2. Visit the CDC website at https://www.cdc.gov/coronavirus/2019-ncov/index.html.\n3. Visit the FDA website at https://www.fda.gov/emergency-preparedness-and-response/mcm-legal-regulatory-and-policy-framework/emergency-use-authorization.\n4. Contact your local or state public health department for guidance and information.\n\nYou can also visit the global website www.cvdvaccine.com or call the telephone number 1-877-829-2619 (1-877-VAX-CO19) for additional information.'

In [27]:
query = """Act as a university student and write a paragraph in an academic writing style on why it is important to take the COVID-19 vaccine.
include the following points:
1. taking the vaccine reduces symptoms of COVID-19
2. taking the vaccine reduces the spread of COVID-19
3. the vaccine is free to take and has minor side effects"""
result = chain({"question": query})
result["answer"]

"Taking the COVID-19 vaccine, such as COMIRNATY (COVID-19 Vaccine, mRNA) or the Pfizer-BioNTech COVID-19 Vaccine, has several benefits:\n\n1. Reducing symptoms: The vaccine can help reduce the severity of COVID-19 symptoms if you contract the virus. It can also lower the risk of hospitalization and death.\n\n2. Preventing the spread of the virus: Getting vaccinated can reduce the transmission of the virus to others. By getting vaccinated, you contribute to community immunity, also known as herd immunity, which helps protect those who cannot get vaccinated or have a weaker immune response.\n\n3. Minimal side effects: The vaccines have been extensively studied and have shown to have a high safety profile. Common side effects, such as pain at the injection site, fatigue, and mild flu-like symptoms, are usually temporary and resolve within a few days. Severe side effects are rare.\n\n4. Free availability: The COVID-19 vaccines are available at no cost to individuals in many countries, incl

In [28]:
query = """Act as a town mayor and write a paragraph in an emotive writing style on why it is important to take the COVID-19 vaccine.
include the following points:
1. taking the vaccine reduces symptoms of COVID-19
2. taking the vaccine reduces the spread of COVID-19
3. the vaccine is free to take and has minor side effects"""
result = chain({"question": query})
result["answer"]

'It is important for individuals to take the COVID-19 vaccine because it can help prevent COVID-19 and its associated symptoms. The vaccine can also reduce the spread of the virus, protecting not only the vaccinated individual but also those around them, especially vulnerable populations. While the vaccine may have minor side effects, they are generally outweighed by the benefits of vaccination. It is recommended to discuss any concerns or questions about the vaccine with a healthcare provider.'

In [30]:
query = """Act as a vaccine provider and write paragraph in an explanatory writing style on why it is important to take the COVID-19 vaccine.
include the following points:
1. taking the vaccine reduces symptoms of COVID-19
2. taking the vaccine reduces the spread of COVID-19
3. the vaccine is free to take and has minor side effects"""
result = chain({"question": query})
result["answer"]

'The fact sheet states that taking the COVID-19 vaccine is important because it can help prevent COVID-19 and its associated symptoms. It is also mentioned that getting vaccinated may help protect those around you, including your close contacts. Taking the vaccine is a way to contribute to efforts to control and reduce the spread of the virus during the current pandemic.'